In [1]:
import pandas as pd
import re
import requests
import json
import cloudscraper
import time
import logging
import sqlite3

In [2]:
logging.basicConfig(filename = 'Crawler.log',level=logging.DEBUG, format='%(levelname)s:%(asctime)s')

# Step 1. Extract Data

### Beacause the Dcard used Cloudflare to protect their server this year, so i use the library 'cloudscraper' to replace ' requests'.   

### And this new package is a free version, I only take five rows of data here for display.

In [3]:
scraper = cloudscraper.create_scraper()
# r = scraper.get('https://www.dcard.tw/_api/forums/apple/posts&limit=100').text 
r = scraper.get('https://www.dcard.tw/service/api/v2/forums/apple/posts?popular=true&limit=5').text 
y = json.loads(r)
print (y)

[{'id': 235130101, 'title': '#Mac M1 挑選指南總整理', 'excerpt': '2021/06/22 新增分析內文於新文章內，#Mac 21年中的M1挑選再分析，️如果想要留言詢問的，請參閱新篇並在新文章底下留言即可️，兩篇都是我…會回覆解答。＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝又見面了，', 'anonymousSchool': False, 'anonymousDepartment': False, 'pinned': True, 'forumId': '42bfdc63-8601-4e87-b53c-53d831caa2cf', 'replyId': None, 'createdAt': '2021-01-07T09:04:46.469Z', 'updatedAt': '2021-07-10T08:34:24.873Z', 'commentCount': 306, 'likeCount': 2456, 'withNickname': False, 'tags': ['HIDE_THUMBNAIL'], 'topics': ['Apple', '3c', 'MacBook', '筆電', 'MacBookPro'], 'meta': {'layout': 'link'}, 'forumName': 'Apple', 'forumAlias': 'apple', 'nsfw': False, 'gender': 'M', 'school': '中國文化大學', 'department': '大氣科學系', 'replyTitle': None, 'mediaMeta': [], 'reactions': [{'id': '286f599c-f86a-4932-82f0-f5a06f1eca03', 'count': 2450}, {'id': 'e8e6bc5d-41b0-4129-b134-97507523d7ff', 'count': 3}, {'id': '4b018f48-e184-445f-adf1-fc8e04ba09b9', 'count': 2}, {'id': 'aa0d425f-d530-4478-9a77-fe3aedc79eea', 'count': 1}], 'hidden': False, 'custom

In [4]:
y[3]

{'id': 233525109,
 'title': '#分享 #果粉必看 iPHONE SE 二代 平價新機差異比較全攻略！預購通路報給你知！',
 'excerpt': '打給厚胎嘎厚，挖係徐自摸啦！，前幾天新款的 iPHONE SE 二代 上市之後，許多果粉議論紛紛的在思考到底值不值得替換手上的裝置，還在猶豫的同學可以先看完這篇 iPhone SE 二代、iPhone',
 'anonymousSchool': False,
 'anonymousDepartment': False,
 'pinned': True,
 'forumId': '42bfdc63-8601-4e87-b53c-53d831caa2cf',
 'replyId': None,
 'createdAt': '2020-04-21T19:34:47.761Z',
 'updatedAt': '2020-12-08T04:55:37.786Z',
 'commentCount': 75,
 'likeCount': 617,
 'withNickname': True,
 'tags': [],
 'topics': ['新機', 'Apple', 'iPhone', '差異', '蘋果'],
 'meta': {'layout': 'classic'},
 'forumName': 'Apple',
 'forumAlias': 'apple',
 'nsfw': False,
 'gender': 'M',
 'school': '大安徐志摩',
 'department': 'minxiongtw',
 'replyTitle': None,
 'mediaMeta': [{'id': '2ca0d79b-abf8-4c46-94a5-7bffc15da6b0',
   'url': 'https://imgur.dcard.tw/6lerlzM.jpg',
   'normalizedUrl': 'https://imgur.dcard.tw/6lerlzM.jpg',
   'thumbnail': 'https://imgur.dcard.tw/6lerlzM.jpg',
   'type': 'image/thumbnail',
   'tags': ['ANNOTAT

In [5]:
def Crawler(ID):
    print(str(ID))
    link = 'https://www.dcard.tw/_api/posts/' + str(ID)
    requ = scraper.get(link)
    rejs = requ.json()
    mediaURLlist = []
    if rejs['media']:
        for i in range(len(rejs['media'])):
            mediaURLlist.append(rejs['media'][i]['url'])
            
    return(pd.DataFrame(
        data=[{'ID':rejs['id'],
          'title':rejs['title'],
          'mediaURL':','.join(mediaURLlist),
          'categories':rejs['categories'],
          'content':rejs['content'],
          'createdAt':rejs['createdAt'],
          'updatedAt':rejs['updatedAt'],
          'topics':rejs['topics']}],
        columns=['ID','title','mediaURL','categories','content','createdAt','updatedAt','topics']))

In [6]:
# Check function
Crawler(236727071)

236727071


,ID,title,mediaURL,categories,content,createdAt,updatedAt,topics
0,236727071,#iPad 請益 有沒有人用亞果type c六合一,"https://i.imgur.com/QldmCW5.jpg,https://i.imgu...",[iPad],這款價錢看起來很划算，不知道為什麼都沒有人推薦他？想請問有沒有使用心得，謝謝大家！\n因為他...,2021-08-15T09:06:10.779Z,2021-08-15T09:06:10.779Z,[typechub]


In [7]:
def get_apple(start, hr_interval= 6, roundOut=5):
    
    interval = hr_interval*60*60  # hr * 60min * 60 second , the default interval time is 6 hours.
    e_time = start + interval*5
    df = pd.DataFrame()
    next_round_start_id = None
    
    while True:
        if roundOut == 0 or time.time() > e_time:
            break
        
        if not next_round_start_id:
            apple_url = 'https://www.dcard.tw/service/api/v2/forums/apple/posts?&limit=7'
        else:  
            apple_url = 'https://www.dcard.tw/service/api/v2/forums/apple/posts?&limit=7&before=' +str(next_round_start_id)
        
        apple_scraper = cloudscraper.create_scraper()
        apple_re =  apple_scraper.get(apple_url).text 
        apple_js = json.loads(apple_re)
        for i in range(len(apple_js)):
            try:
                if i == len(apple_js)-1:
                    next_round_start_id = apple_js[i]['id'] # Save the last data id, and be used for to get next data in next round
                df = df.append(Crawler(apple_js[i]['id']),ignore_index=True)
                time.sleep(1)
            except:    # if meet errors, write the log file.
                print('special id: ',apple_js[i]['id'],' meet some error')
                logging.warning('special id: ',apple_js[i]['id'],' meet some error')
        roundOut -= 1
    return df

# Step 2. Extract the information from Dcard website.

### In order to avoid the problem of charging, so set a parameter 'roundOut' to interrupt execution.

In [8]:
s_time = time.time()
result_table = get_apple(s_time,roundOut=1)

236730323
236730300
236729851
236729767
236729676
236729591
236729374


In [9]:
result_table.head()

,ID,title,mediaURL,categories,content,createdAt,updatedAt,topics
0,236730323,#iPhone 東森買iPhone 安全嗎？,,[iPhone],不好意思，最近想請問一下在東森購物網買iPhone 是正品嗎？\n有查了一下，有看到別人說基...,2021-08-15T16:29:08.060Z,2021-08-15T16:29:08.060Z,"[東森, iPhone, Apple, 購物, 安全]"
1,236730300,#iPad 平板包選擇,"https://i.imgur.com/98vTRiV.jpg,https://i.imgu...",[iPad],大家晚安～\n想必最近很多人都在買各式各樣的平板週邊商品\n我在挑選平板包的時候真的選擇困難...,2021-08-15T16:25:18.536Z,2021-08-15T16:25:18.536Z,"[平板, iPad, Ipadair4, 配件]"
2,236729851,#iPad Ipad突然黑屏又重開,,[iPad],我的型號是Ipad Air3，之前很偶爾會出現突然黑屏且有白色的圈不斷在跑，過一下下就會出現...,2021-08-15T15:28:55.311Z,2021-08-15T15:28:55.311Z,"[黑屏, iPad, 筆電]"
3,236729767,#iPad 請益ipad pro 11螢幕保護貼,,[iPad],大家好\n這幾天爬了許多保護殼和貼的文\n有些保護貼和殼吃邊的細節仍然不太清楚\n想請問一下...,2021-08-15T15:19:22.074Z,2021-08-15T15:22:14.470Z,"[iPad, 螢幕]"
4,236729676,#iPhone iPhone音樂,,[iPhone],想請問就是iPhone音樂只能用內建的apple music 才可以聽嗎 因為我從別的地方下...,2021-08-15T15:07:24.116Z,2021-08-15T15:07:24.116Z,"[音樂, iPhone]"


In [10]:
result_table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   ID          7 non-null      int64 
 1   title       7 non-null      object
 2   mediaURL    7 non-null      object
 3   categories  7 non-null      object
 4   content     7 non-null      object
 5   createdAt   7 non-null      object
 6   updatedAt   7 non-null      object
 7   topics      7 non-null      object
dtypes: int64(1), object(7)
memory usage: 576.0+ bytes


### The data in the table must be converted into a string type, before it can be stored in the database.

In [11]:
result_table = result_table.applymap(str)
result_table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   ID          7 non-null      object
 1   title       7 non-null      object
 2   mediaURL    7 non-null      object
 3   categories  7 non-null      object
 4   content     7 non-null      object
 5   createdAt   7 non-null      object
 6   updatedAt   7 non-null      object
 7   topics      7 non-null      object
dtypes: object(8)
memory usage: 576.0+ bytes


# Step 3. Prpare to write into database

### Here, i choose to use SQLite to save data.

In [12]:
conn = sqlite3.connect('CrawlerTable.db')  # build database
cursor = conn.cursor()
cursor.execute('CREATE TABLE CrawlerTable(ID, title, mediaURL, categories, content,createdAt,updatedAt,topics)')  #建立資料表
conn.commit()

### Write the data into the database, if the database doesn't exist, create a new one, otherwise continue to write into the existing one.

In [13]:
result_table.to_sql('CrawlerTable', conn, if_exists='append', index=False) 

### Check whether the data in the database is normal

In [14]:
check_df = pd.read_sql("SELECT * FROM CrawlerTable", conn)
check_df

,ID,title,mediaURL,categories,content,createdAt,updatedAt,topics
0,236730323,#iPhone 東森買iPhone 安全嗎？,,['iPhone'],不好意思，最近想請問一下在東森購物網買iPhone 是正品嗎？\n有查了一下，有看到別人說基...,2021-08-15T16:29:08.060Z,2021-08-15T16:29:08.060Z,"['東森', 'iPhone', 'Apple', '購物', '安全']"
1,236730300,#iPad 平板包選擇,"https://i.imgur.com/98vTRiV.jpg,https://i.imgu...",['iPad'],大家晚安～\n想必最近很多人都在買各式各樣的平板週邊商品\n我在挑選平板包的時候真的選擇困難...,2021-08-15T16:25:18.536Z,2021-08-15T16:25:18.536Z,"['平板', 'iPad', 'Ipadair4', '配件']"
2,236729851,#iPad Ipad突然黑屏又重開,,['iPad'],我的型號是Ipad Air3，之前很偶爾會出現突然黑屏且有白色的圈不斷在跑，過一下下就會出現...,2021-08-15T15:28:55.311Z,2021-08-15T15:28:55.311Z,"['黑屏', 'iPad', '筆電']"
3,236729767,#iPad 請益ipad pro 11螢幕保護貼,,['iPad'],大家好\n這幾天爬了許多保護殼和貼的文\n有些保護貼和殼吃邊的細節仍然不太清楚\n想請問一下...,2021-08-15T15:19:22.074Z,2021-08-15T15:22:14.470Z,"['iPad', '螢幕']"
4,236729676,#iPhone iPhone音樂,,['iPhone'],想請問就是iPhone音樂只能用內建的apple music 才可以聽嗎 因為我從別的地方下...,2021-08-15T15:07:24.116Z,2021-08-15T15:07:24.116Z,"['音樂', 'iPhone']"
5,236729591,#iPhone airpods犀牛盾 顏色差…..,"https://i.imgur.com/kVktABS.jpg,https://i.imgu...",['iPhone'],最近想買airpods的殼\n原本看到網路上奶茶色的\n感覺蠻好看的\n如圖\nhttps:...,2021-08-15T14:59:16.220Z,2021-08-15T14:59:16.220Z,"['犀牛盾', 'Airpods']"
6,236729374,#Mac Steam下載的遊戲怎麼移除😰,"https://i.imgur.com/ONSfCfG.jpg,https://i.imgu...",['Mac'],https://i.imgur.com/ONSfCfG.jpg\n如上圖這種單個程式怎麼從啟...,2021-08-15T14:29:10.992Z,2021-08-15T14:29:10.992Z,"['steam', '移除', 'MacBook']"


# Finally, I have no writed the backfill policy.